In [1]:
from tsai.all import computer_setup

computer_setup()

os              : Linux-6.6.10-76060610-generic-x86_64-with-glibc2.35
python          : 3.12.1
tsai            : 0.3.9
fastai          : 2.7.14
fastcore        : 1.5.29
torch           : 2.2.0
device          : 1 gpu (['NVIDIA GeForce GTX 1070 with Max-Q Design'])
cpu cores       : 6
threads per cpu : 2
RAM             : 31.19 GB
GPU memory      : [8.0] GB


In [2]:
import joblib
import numpy as np
import torch
from torch import tensor

In [3]:
raw_data = (joblib.load("raw_data.job")).real
print(raw_data.shape)

(1000, 5000, 4)


In [4]:
# N = raw_data.shape[0]
# C = raw_data.shape[1]
# L = raw_data.shape[2]
# kernel_window = 4

number_of_features = 4
no_filters = 16


lookback = 5

In [5]:
def create_dataset(dataset, lookback):
    """Transform a time series into a prediction dataset

    Args:
        dataset: A numpy array of time series, first dimension is the time steps
        lookback: Size of window for prediction
    """
    X, y = [], []
    for i in range(len(dataset) - lookback):
        feature = dataset[i : i + lookback]
        target = dataset[i + 1 : i + lookback + 1]
        X.append(feature)
        y.append(target)
    return torch.stack(X), torch.stack(y)

In [6]:
X_complete_windowed_df, y_complete_windowed_df = torch.vmap(
    lambda df: create_dataset(df, lookback=lookback)
)(tensor(raw_data))

print(X_complete_windowed_df.shape)

print(
    "as we see, the first dimension corresponds to the different trajectories. Now we don't care about this anymore, we can merge them."
)

X_complete_windowed_df = X_complete_windowed_df.reshape(
    -1, lookback, number_of_features
)

y_complete_windowed_df = y_complete_windowed_df.reshape(
    -1, lookback, number_of_features
)

print(X_complete_windowed_df.shape)

print("Don't forget to swap 1 and 2 index for cnn network")

torch.Size([1000, 4995, 5, 4])
as we see, the first dimension corresponds to the different trajectories. Now we don't care about this anymore, we can merge them.
torch.Size([4995000, 5, 4])
Don't forget to swap 1 and 2 index for cnn network
